# Berlin City-Wide Emissions - Cleaning and Transformation

This notebook cleans and transforms the dataset `CSV data/2023_City_Wide_Emissions_Berlin.csv`.
Each step includes a markdown explanation and detailed code so you can follow the workflow.

## 1. Imports
Load the required libraries and configure display options for better readability.

In [ ]:

# Core analysis libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Display more columns when inspecting DataFrames
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 140)

# Set a consistent plotting theme
sns.set_theme(style='whitegrid', palette='deep')


## 2. Load the dataset
Read the CSV into a DataFrame, preview the first rows, and record the initial shape.

In [ ]:

# Path to the raw data
DATA_PATH = Path('CSV data') / '2023_City_Wide_Emissions_Berlin.csv'

# Load the dataset
df_raw = pd.read_csv(DATA_PATH)
initial_shape = df_raw.shape

print(f'Loaded data from: {DATA_PATH}')
print(f'Initial shape (rows, columns): {initial_shape}')
df_raw.head()


## 3. Column cleaning
Strip stray whitespace, rename verbose headers, and convert names to snake_case for consistency.

In [ ]:

# Work on a copy so the raw frame is preserved
df = df_raw.copy()

# First trim stray spaces from column names
df.columns = df.columns.str.strip()

# Rename particularly long or ambiguous headers
rename_map = {
    'Primary protocol/framework used to compile main inventory': 'Inventory Protocol',
    'Gases included in main inventory': 'Included Gases',
    'Emissions Question Name': 'Emissions Question',
    'Emissions Column Name': 'Emissions Metric',
    'Emissions Row Name': 'Emissions Category',
    'Emissions Response Answer': 'Emissions Value',
    'Emissions Notation Key': 'Emissions Notation',
    'Emissions Description': 'Emissions Notes',
    'Emissions Data Group': 'Emissions Group',
    'Year covered by main inventory': 'Inventory Year',
    'Population in year covered by main inventory': 'Inventory Population',
    'Boundary of main inventory relative to jurisdiction boundary': 'Inventory Boundary',
    'Tool used to compile main inventory': 'Inventory Tool',
    'City Location': 'City Location WKT'
}
df = df.rename(columns=rename_map)

# Convert column names to snake_case
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(r'[^0-9a-z]+', '_', regex=True)
      .str.replace(r'_+', '_', regex=True)
      .str.strip('_')
)

print('Cleaned column names:')
df.columns.to_list()


## 4. Data type conversion
Convert important fields (year, emissions, population, etc.) to numeric types.
Errors are coerced to NaN so unexpected strings become easy to flag.

In [ ]:

# Columns that should be numeric after cleaning
numeric_like_cols = [
    'inventory_year',
    'emissions_value',
    'inventory_population',
    'organization_number',
    'number_of_times_reporting',
    'emissions_rank'
]

for col in numeric_like_cols:
    if col in df.columns:
        # Remove commas and extra spaces before conversion
        df[col] = (
            df[col]
              .astype(str)
              .str.replace(',', '', regex=False)
              .str.strip()
        )
        df[col] = pd.to_numeric(df[col], errors='coerce')

df[numeric_like_cols].dtypes


## 5. Missing value handling
Review null counts, fill the population (single value across rows), and drop records that lack emissions data.

In [ ]:

# Review missing values
missing_counts = df.isna().sum().sort_values(ascending=False)
print('Columns with missing values:')
display(missing_counts[missing_counts > 0])

# Inventory population is constant for the year; forward fill with that single value
if 'inventory_population' in df.columns:
    population_values = df['inventory_population'].dropna().unique()
    if population_values.size == 1:
        df['inventory_population'] = df['inventory_population'].fillna(population_values[0])

# Remove rows that do not have an emissions measurement
before_drop = df.shape[0]
df = df.dropna(subset=['emissions_value'])
dropped_rows = before_drop - df.shape[0]
print(f'Dropped {dropped_rows} rows lacking emissions values.')

df.isna().sum().sort_values(ascending=False).head()


## 6. Filter to Berlin and select relevant columns
Focus on Berlin rows, derive sector and scope, and keep the columns needed for analysis.

In [ ]:

# Keep only the City of Berlin
df = df[df['city'].str.strip().str.lower() == 'berlin'].copy()
print(f'Remaining rows after filtering for Berlin: {df.shape[0]}')

# Extract sector information from the hierarchical category
if 'emissions_category' in df.columns:
    df['sector'] = (
        df['emissions_category']
          .str.replace('^', '', regex=False)
          .str.split('>')
          .str[0]
          .str.strip()
    )
    df['category_path'] = df['emissions_category'].str.replace('^', '', regex=False).str.strip()

# Map textual descriptions to scope buckets
scope_map = {
    'direct emissions': 'Scope 1',
    'indirect emissions': 'Scope 2',
    'outside the jurisdiction boundary': 'Scope 3'
}
df['scope'] = 'Other/Unknown'
for phrase, label in scope_map.items():
    df.loc[df['emissions_metric'].str.contains(phrase, case=False, na=False), 'scope'] = label
df['scope_category'] = df['scope']

# Create harmonised aliases used later in the workflow
df['year'] = df['inventory_year'].astype('Int64')
df['emissions_tco2e'] = df['emissions_value']
df['population'] = df['inventory_population']
df['gas_type'] = df.get('included_gases', np.nan)

# Keep the most relevant columns for analysis
priority_columns = [
    'city', 'country', 'year', 'emissions_tco2e', 'sector', 'category_path',
    'scope', 'scope_category', 'gas_type', 'population', 'emissions_group',
    'emissions_metric', 'emissions_question'
]
existing_columns = [col for col in priority_columns if col in df.columns]
df = df[existing_columns]
df.head()


## 7. Remove duplicate records
Drop any duplicated rows to avoid double counting emissions.

In [ ]:

before_dedup = df.shape[0]
df = df.drop_duplicates()
removed = before_dedup - df.shape[0]
print(f'Removed {removed} duplicate rows.')
df.shape


## 8. Feature engineering
Create per-capita emissions and tidy the scope category column.

In [ ]:

# Compute per-capita emissions where population data is available
if {'emissions_tco2e', 'population'}.issubset(df.columns):
    df['emissions_per_capita'] = np.where(
        (df['population'] > 0) & ~df['population'].isna(),
        df['emissions_tco2e'] / df['population'],
        np.nan
    )
else:
    df['emissions_per_capita'] = np.nan

# Ensure scope_category is populated (mirrors scope by design)
df['scope_category'] = df['scope_category'].fillna(df['scope'])
df.head()


## 9. Sort, reset index, and export
Order the cleaned data, reset the index, and save to a new CSV for downstream analysis.

In [ ]:

cleaned_path = Path('CSV data') / '2023_City_Wide_Emissions_Berlin_Cleaned.csv'

df = df.sort_values(by=['year', 'sector', 'category_path'], ascending=[True, True, True])
df = df.reset_index(drop=True)

df.to_csv(cleaned_path, index=False)
print(f'Saved cleaned data to: {cleaned_path.resolve()}')
df.head()


## 10. Summary checks
Compare shapes before and after cleaning, inspect any remaining nulls, and preview key fields.

In [ ]:

    final_shape = df.shape
    print(f'Initial shape : {initial_shape}')
    print(f'Final shape    : {final_shape}')

    print('
Missing values after cleaning:')
    display(df.isna().sum()[df.isna().sum() > 0])

    key_columns = [col for col in ['sector', 'scope_category', 'year'] if col in df.columns]
    for col in key_columns:
        uniques = df[col].dropna().unique()
        print(f"
Unique values for {col!r} ({len(uniques)} found):")
        print(uniques)

    df.sample(min(5, len(df)))


## 11. Quick visualisations
Produce a few simple charts to sanity check the cleaned data: totals by sector and scope,
plus a trend over time.

In [ ]:

# Ensure plots render inline when using Jupyter
%matplotlib inline

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Bar chart: total emissions per sector
if 'sector' in df.columns:
    sector_totals = df.groupby('sector')['emissions_tco2e'].sum().sort_values(ascending=False)
    sector_totals.plot(kind='bar', ax=axes[0])
    axes[0].set_title('Total Emissions by Sector')
    axes[0].set_ylabel('tCO2e')
else:
    axes[0].set_visible(False)

# Pie chart: emissions split by scope
if 'scope_category' in df.columns:
    scope_totals = df.groupby('scope_category')['emissions_tco2e'].sum()
    axes[1].pie(scope_totals, labels=scope_totals.index, autopct='%1.1f%%')
    axes[1].set_title('Emissions Share by Scope')
else:
    axes[1].set_visible(False)

# Line plot: emissions trend over time
if 'year' in df.columns:
    yearly_totals = df.groupby('year')['emissions_tco2e'].sum().sort_index()
    axes[2].plot(yearly_totals.index, yearly_totals.values, marker='o')
    axes[2].set_title('Total Emissions Over Time')
    axes[2].set_ylabel('tCO2e')
    axes[2].set_xlabel('Year')
else:
    axes[2].set_visible(False)

plt.tight_layout()
plt.show()
